[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/y-akbal/Tedu_Computational_Statistics/blob/main/11/knn_test.ipynb)

## Bootstrap Error

In [1]:
import sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import concurrent
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [2]:
X, y = load_digits(return_X_y = True)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [7]:
knn = KNeighborsClassifier()

In [8]:
def return_bootstrap(X_train, y_train):
    len_ = X_train.shape[0]
    indices = np.array([i for i in range(len_)])
    indice = np.random.choice(indices, size = len(indices),replace = True).reshape(-1)
    return X_train[indice], y_train[indice], indice
X_train_b, y_train_b, indice = return_bootstrap(X_train, y_train)

In [9]:
L = []
for i in range(1000):
    X_train_b, y_train_b, indice = return_bootstrap(X_train, y_train)
    knn.fit(X_train_b, y_train_b)
    L.append(knn.score(X_test, y_test))

In [10]:
np.mean(L)

0.9840133333333333

In [11]:
R = []
knn = KNeighborsClassifier(n_neighbors= 10)
for i in range(1000):
    X_train_b, y_train_b, indice = return_bootstrap(X_train, y_train)
    knn.fit(X_train_b, y_train_b)
    R.append(knn.score(X_test, y_test))

In [83]:
np.mean(R)

0.9754688888888889

## One-leave-out error:

In [35]:
def one_leave_out_indice(X_train, y_train):
    assert len(X_train) == len(y_train)
    N = X_train.shape[0]
    
    indice = [i for i in range(N)]
    for i in range(1, N-1):
        train, test = np.array(indice[:i]+indice[i+1:], dtype = np.int32), np.array([i])
        yield X_train[train], y_train[train], X_train[test], y_train[test]

In [39]:
R = []
gen = one_leave_out_indice(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors= 3)
for X_train, y_train, X_test, y_test in gen:
    knn.fit(X_train, y_train)
    R.append(knn.score(X_test, y_test))

In [41]:
np.mean(R)

0.9851079672375279

In [42]:
R = []
gen = one_leave_out_indice(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors= 7)
for X_train, y_train, X_test, y_test in gen:
    knn.fit(X_train, y_train)
    R.append(knn.score(X_test, y_test))
np.mean(R)

0.981371087928465

# K - fold cross-validation

In [51]:
def k_fold_indices(X_train, y_train, fold = 5):
    # your code